In [13]:
import pandas as pd
import numpy as np
import gzip
import pickle

In [14]:
with gzip.open(r'../data/LibData.pkl.gz') as f:
    library_data = pickle.load(f)

In [15]:
comp_attrs = pd.read_csv(r'../data/computerAttributes.csv',header=0)

In [16]:
grouped_names = comp_attrs[comp_attrs['requiresLogon'] == True].set_index(['inJackson','floor','location']).sort_index()['computerName'].values

In [17]:
sem_table = pd.read_csv(r'../data/semesters.csv',header=0)

In [18]:
from bokeh.io import curdoc
from bokeh.layouts import row, column, widgetbox
from bokeh.models import( 
                Spacer, 
                FuncTickFormatter, 
                FixedTicker, 
                HoverTool, 
                ColumnDataSource, 
                LinearColorMapper,
                ColorBar, 
                BasicTicker, 
                PrintfTickFormatter)
from bokeh.plotting import figure, output_file, output_notebook, show, save
from bokeh.models.widgets import Slider
from bokeh.palettes import inferno

hover = HoverTool(
    tooltips=[
        ("Computer", "@comps"),
        ("Hour", "$y{0}:00"),
        ("Pct Use","@means")
    ],
    formatters={"Hour":"datetime"}
)

TOOLS=[hover,"crosshair,pan,wheel_zoom,box_zoom,reset,tap,save,box_select"]
mapper = LinearColorMapper(palette=inferno(10), low=0, high=100)
output_notebook()

Loading BokehJS ...

In [19]:
row_id = 21
sem_sdate = sem_table.loc[row_id,'startDate']
sem_edate = sem_table.loc[row_id,'endDate']
sem_semst = sem_table.loc[row_id,'semester']
date_mask = (library_data.index >= sem_sdate) & (library_data.index <= sem_edate)
sem_avgs = library_data[date_mask].groupby(library_data[date_mask].index.hour).mean()*100

In [20]:
means_unstack = sem_avgs.loc[:,grouped_names].unstack().reset_index()
means_unstack.columns = ["comps","hour","means"]
means_unstack['x_vals'] = means_unstack.index / 24 * 24

In [21]:
map_source = ColumnDataSource.from_df(means_unstack)

In [22]:
mainGraph = figure(tools=TOOLS, plot_width=900, plot_height=500, x_range=(0, 7000), y_range=(0, 23),
                     min_border=10, min_border_left=20,
                     toolbar_location="above",
                     x_axis_location=None, # this is left in, as the x-axis ticks are hard to read zoomed out.
                     #y_axis_location=None, 
                     title="Library Usage: Average Percent Utilization per Hour ("+sem_semst+")")
mainGraph.background_fill_color = "#fafafa"
# mainGraph.select(BoxSelectTool).select_every_mousemove = False
# mainGraph.select(LassoSelectTool).select_every_mousemove = False
mainGraph.yaxis.formatter = FuncTickFormatter(code="""return Math.floor(tick)+':00'""")
mainGraph.yaxis.ticker = FixedTicker(ticks = range(0,24))
mainGraph.rect(x="x_vals", y="hour", 
               width=24, height=1,
               source=map_source,
               fill_color={'field': 'means', 'transform': mapper},
              line_color=None)
# output_file("./AvgPercentUtil.html", title='Library Usage: Average Percent Utilization per Hour')
color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="8pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=10, border_line_color=None, location=(0, 0))
mainGraph.add_layout(color_bar, 'right')

In [23]:
sem_slide = Slider(start=0, end=24, value=21, step=1, title="Semester")

inputs = widgetbox(sem_slide)
show(column(inputs,mainGraph))